<a href="https://colab.research.google.com/github/hieulv416c/Url_content_detect/blob/main/ELMo_%2B_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
lehieu12_cleaned_data_url68k_path = kagglehub.dataset_download('lehieu12/cleaned-data-url68k')

print('Data source import complete.')


In [ ]:
import threading
import time

def keep_alive():
    while True:
        print("Keeping the session alive...")
        time.sleep(1800)  # 10 phút

thread = threading.Thread(target=keep_alive, daemon=True)
thread.start()


Keeping the session alive...


In [ ]:
!pip install --upgrade gensim
!pip install --upgrade pip
!pip install catboost
!pip install xgboost
import gensim
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np


import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Load data
file_path = '/kaggle/input/cleaned-data-url68k/content_cleaned.csv'

df = pd.read_csv(file_path)
print(df.head())


df = df.dropna()



df["Content"] = df["Content"].astype(str)

import os
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# 🚀 **Chạy trên CPU để tránh lỗi OOM**
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# 📌 **Tải mô hình ELMo từ TensorFlow Hub**
elmo_model = hub.load("https://tfhub.dev/google/elmo/3")

# ✅ **Hàm chuyển đổi văn bản thành ELMo embedding**
def elmo_embedding(texts, batch_size=1):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i : i + batch_size]

        # 📌 **Fix lỗi: Dùng "default" và truyền trực tiếp text vào model**
        with tf.device('/CPU:0'):
            elmo_output = elmo_model.signatures["default"](tf.constant(batch_texts))

        # 📌 **Lấy vector trung bình của các từ trong câu**
        batch_embeddings = tf.reduce_mean(elmo_output["elmo"], axis=1).numpy()
        embeddings.append(batch_embeddings)

    return np.vstack(embeddings)


# ✅ **Chia tập train/test**
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["Content"].tolist(), df["result"].tolist(), test_size=0.2, random_state=42)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.2 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
       File Name                                              Title  \
0  10000847.html  Wells Fargo – Banking, Credit Cards, Loans, Mo...   
1  10000855.html  La Banque Postale - Banque et Assurance en lig...   
2  10000856.html                                   LCL - Mon espace   
3  10000946.html                                           No title   
4  10000976.html                                           Facebook   

                                             Content  \
0  well fargo banking credit card loan mortgage s...   
1  la banque postale banque et assurance en ligne...   
2  lcl mon espace votre identifiant mémoriser mon...   
3  ov5qtm_ydkxfcuwsmak29bb94o4mxy5n3mpm62bkahmjda...   
4  facebook messenger albin berisha albin shared ...   

           

In [ ]:


# ✅ **Chuyển đổi văn bản thành embedding ELMo**
X_train = elmo_embedding(train_texts, batch_size=16)  # Batch nhỏ để tránh OOM
X_test = elmo_embedding(test_texts, batch_size=16)
y_train = np.array(train_labels, dtype=np.int64)
y_test = np.array(test_labels, dtype=np.int64)

# # ✅ **Chuẩn hóa dữ liệu cho các mô hình cần**
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# 📌 **Danh sách mô hình ML**
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
}

# # ✅ **Danh sách các mô hình CẦN chuẩn hóa**
# models_need_scaling = ["SVM", "Logistic Regression", "KNN", "MLP Classifier"]


In [ ]:
# 📌 **Huấn luyện và đánh giá từng mô hình**
for name, model in models.items():
    print(f"\n🚀 Đang huấn luyện mô hình: {name}...")

    # 🔹 **Dùng dữ liệu đã chuẩn hóa cho các mô hình cần**
    # if name in models_need_scaling:
    #     model.fit(X_train_scaled, y_train)
    #     y_pred = model.predict(X_test_scaled)
    # else:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # 📌 **Tính các chỉ số đánh giá**
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="weighted")
    recall = recall_score(y_test, y_pred, average="weighted")
    f1 = f1_score(y_test, y_pred, average="weighted")

    print(f"🎯 Accuracy: {accuracy:.4f}")
    print(f"🎯 Precision: {precision:.4f}")
    print(f"🎯 Recall: {recall:.4f}")
    print(f"🎯 F1-score: {f1:.4f}")
    print("📊 Báo cáo phân loại:")
    print(classification_report(y_test, y_pred))

    # 📌 **Vẽ ma trận nhầm lẫn**
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
                xticklabels=["Không Phishing", "Phishing"], yticklabels=["Không Phishing", "Phishing"])
    plt.xlabel("Dự đoán")
    plt.ylabel("Thực tế")
    plt.title(f"🔍 Ma trận nhầm lẫn - {name}")
    plt.show()

print("🎉 Quá trình huấn luyện và đánh giá mô hình hoàn tất!")